Neste notebook irei mostrar a prepação dos dados que usei para fazer o dashboard no Power BI

OBS: Ao final das células de código irei deixar uma breve explicação do significado de cada coluna que eu selecionei para fazer o processo de ETL no Power BI

#Preparação Dos Dados Em Passos

##Primeiro Passo: Vamos importa a biblioteca que tem as informações que precisamos e vamos importar também a biblioteca que vai permitir a manipulação dos dados

In [1]:
!pip install fundamentus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.5 MB/s eta 0:00:00


In [2]:
import fundamentus
import pandas as pd
import warnings #Esse warnings eu utilizei para poder solucionar por um erro que a base de dados dava ao importar para o Power BI
warnings.filterwarnings("ignore", category=FutureWarning)

##Segundo Passo: Vamos definir nossa carteira de ações, ou seja, as ações que iremos utilizar

In [3]:
carteira_fund = ['BBAS3', 'BBDC4', 'BBSE3', 'CMIG4', 'ITSA4',
                 'KLBN11', 'TAEE11', 'TASA4', 'VLID3']

##Terceiro passo: Algumas ações da lista não possuem todas as colunas selecionadas, por isso irei fazer um filtro usando um laço for e depois adicionar esse laço a um DataFrame, e após isso, concatenar esse DF com o que eu vou utilizar para manipular o resto do meu Script

In [4]:
frames = []
for papel in carteira_fund:
    df = fundamentus.get_papel(papel)

    # Selecionando apenas as colunas que estão presentes no DataFrame
    selected_columns = ['Setor', 'Cotacao', 'Min_52_sem', 'Max_52_sem', 'Valor_de_mercado',
                        'Nro_Acoes', 'Patrim_Liq', 'Lucro_Liquido_12m', 'Lucro_Liquido_3m']

    # Verificar se as colunas de receita líquida estão no DataFrame
    if 'Receita_Liquida_12m' in df.columns:
        selected_columns.append('Receita_Liquida_12m')
    if 'Receita_Liquida_3m' in df.columns:
        selected_columns.append('Receita_Liquida_3m')

    frames.append(df[selected_columns])

# Concatenando os DataFrames
ind = pd.concat(frames)

##Quarto Passo: Resetar o index do DF para que o indice seja números e não os nomes dos ativos, e, trocando o nome da coluna index para Ativo, pois, a coluna que apresenta nossas ações merece um nome melhor, não é mesmo?

In [5]:
ind = ind.reset_index()
ind.rename(columns = {'index':'Ativo'}, inplace=True)

##Quinto Passo: Alterar os tipos das colunas de Object para Numeric

In [6]:
ind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Ativo                9 non-null      object
 1   Setor                9 non-null      object
 2   Cotacao              9 non-null      object
 3   Min_52_sem           9 non-null      object
 4   Max_52_sem           9 non-null      object
 5   Valor_de_mercado     9 non-null      object
 6   Nro_Acoes            9 non-null      object
 7   Patrim_Liq           9 non-null      object
 8   Lucro_Liquido_12m    9 non-null      object
 9   Lucro_Liquido_3m     9 non-null      object
 10  Receita_Liquida_12m  7 non-null      object
 11  Receita_Liquida_3m   7 non-null      object
dtypes: object(12)
memory usage: 992.0+ bytes


In [7]:
# Alterando colunas object para numeric
colunas = ['Cotacao', 'Min_52_sem', 'Max_52_sem', 'Valor_de_mercado', 'Nro_Acoes', 'Patrim_Liq',
           'Receita_Liquida_12m', 'Receita_Liquida_3m', 'Lucro_Liquido_12m', 'Lucro_Liquido_3m']
ind[colunas] = ind[colunas].apply(pd.to_numeric, errors='coerce', axis=1)

##Sexto Passo: Criar um novo DataFrame com indicadores para montar nossa Dashboard no Power BI

In [8]:
ind_2 = fundamentus.get_resultado_raw().reset_index()
ind_2 = ind_2.query("papel in @carteira_fund")
ind_2 = ind_2[['papel','P/L', 'Div.Yield','P/VP','ROE']].reset_index(drop=True)
ind_2.rename(columns={'papel': 'Ativo','Div.Yield':'DY'}, inplace= True)

##Sétimo Passo: Usando o método merge, concatenei os dfs em um só com as infos e indicadores

In [9]:
indicadores = pd.merge(ind, ind_2, on="Ativo")

##Oitavo Passo: Criando uma coluna para LPA (Lucro por Ação) e VPA (Valor Patrimonial por ação), para calcular a fórmula de Graham  Valor intrínseco de uma ação (VI = √22,5 x LPA x VPA)

In [10]:
indicadores["LPA"] = (indicadores["Lucro_Liquido_12m"] / indicadores ["Nro_Acoes"]).round(2)
indicadores["VPA"] = (indicadores["Patrim_Liq"] / indicadores ["Nro_Acoes"]).round(2)

del ind, ind_2, carteira_fund, colunas

#Preparação Do Código Em Apenas Uma Única Célula

In [11]:
# Importando as bibliotecas
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import fundamentus
import pandas as pd

# Definindo a carteira de ações
carteira_fund = ['BBAS3', 'BBDC4', 'BBSE3', 'CMIG4', 'ITSA4',
                 'KLBN11', 'TAEE11', 'TASA4', 'VLID3']

# Criando um df com algumas infos da carteira
frames = []
for papel in carteira_fund:
    df = fundamentus.get_papel(papel)

    # Selecionando apenas as colunas que estão presentes no DataFrame
    selected_columns = ['Setor', 'Cotacao', 'Min_52_sem', 'Max_52_sem', 'Valor_de_mercado',
                        'Nro_Acoes', 'Patrim_Liq', 'Lucro_Liquido_12m', 'Lucro_Liquido_3m']

    # Verificar se as colunas de receita líquida estão no DataFrame
    if 'Receita_Liquida_12m' in df.columns:
        selected_columns.append('Receita_Liquida_12m')
    if 'Receita_Liquida_3m' in df.columns:
        selected_columns.append('Receita_Liquida_3m')

    frames.append(df[selected_columns])

# Concatenando os DataFrames
ind = pd.concat(frames)

# Passando o ticker para uma coluna
ind = ind.reset_index()
ind.rename(columns = {'index':'Ativo'}, inplace=True)

# Alterando colunas object para numeric
colunas = ['Cotacao', 'Min_52_sem', 'Max_52_sem', 'Valor_de_mercado', 'Nro_Acoes', 'Patrim_Liq',
           'Receita_Liquida_12m', 'Receita_Liquida_3m', 'Lucro_Liquido_12m', 'Lucro_Liquido_3m']
ind[colunas] = ind[colunas].apply(pd.to_numeric, errors='coerce', axis=1)

# Criando um novo df com alguns indicadores da carteira
ind_2 = fundamentus.get_resultado_raw().reset_index()
ind_2 = ind_2.query("papel in @carteira_fund")
ind_2 = ind_2[['papel','P/L', 'Div.Yield','P/VP','ROE']].reset_index(drop=True)
ind_2.rename(columns={'papel': 'Ativo','Div.Yield':'DY'}, inplace= True)

# Concatenando os dfs em um só com as infos e indicadores
indicadores = pd.merge(ind, ind_2, on="Ativo")

# Criando uma coluna para LPA (Lucro por Ação) e VPA (Valor Patrimonial por ação)
# para calcular a fórmula de Graham  Valor intrínseco de uma ação (VI = √22,5 x LPA x VPA)
indicadores["LPA"] = (indicadores["Lucro_Liquido_12m"] / indicadores ["Nro_Acoes"]).round(2)
indicadores["VPA"] = (indicadores["Patrim_Liq"] / indicadores ["Nro_Acoes"]).round(2)

del ind, ind_2, carteira_fund, colunas

#Algumas informações extras

Antes de eu passar a legendas das colunas do DF, preciso pontuar algumas coisas:

1 - Antes de importar os dados para o Power BI eu criei um ambiente virtual, instalei as bibliotecas lá dentro, e, configurei o Power BI para utilizar o python dentro desse meu ambiente virtual

2 - Assim que carreguei os dados ao Power BI, fiz o processo de ETL, ou seja, eu modifiquei bastante os dados lá dentro antes de criar o dashboard, inclusive até usei Scripts Python dentro do Power BI para deixar o meu dashboard completo

##Legenda Dos Dados

A tabela indicadores dispõe dos seguintes dados:

Ativo: ticker da empresa.

Setor: setor que atua a empresa listada.

Cotacao: cotação de fechamento no último dia.

Min_52_sem: menor cotação do ativo nos últimos 12 meses (52 semanas).

Max_52_sem: maior cotação do ativo nos últimos 12 meses (52 semanas).

Valor_de_mercado: preço pelo qual o mercado está pagando pela empresa.

Receita_Liquida_12m: receita líquida dos últimos 12 meses.

Nro_Acoes: número total de ações.

Patrim_Liq: patrimônio líquido, a diferença entre os ativos e passivos da empresa.

Receita_Liquida_3m: receita líquida dos últimos 3 meses.

Lucro_Liquido_12m: lucro líquido dos últimos 12 meses.

Lucro_Liquido_3m: lucro líquido dos últimos 3 meses.

P/L: preço sobre o lucro; quanto o mercado está disposto a pagar pelos lucros da empresa.

DY: relação entre os dividendos pagos pela empresa (participação dos lucros do investidor) e o preço da ação.

P/VP: preço sobre valor patrimonial; avalia o preço de uma ação entre o seu valor de mercado e o valor que ela tem em patrimônio.

ROE: retorno sobre patrimônio; mede a eficiência de uma empresa na geração de lucro a partir de seus recursos (em %).

LPA: lucro por ação.

VPA: valor patrimonial por ação.